In [1]:
# Import functions #
import argparse,datetime,os,sys,time

import pandas as pd
import plotly.graph_objects as go

from tools import load_groupfile, query_api, filterIQM, merge_dfs, make_vio_plot

In [2]:
# Arguments #

# laziness helper
# here = os.path.dirname(os.path.abspath(os.path.realpath(__file__)))
here = %pwd

# path to input of local data from MRIQC on your own dataset 
group_file = os.path.join(here,'test_data', 'group2_bold.tsv')

# scan type to query the API for [bold, T1w, T2w]
modality = 'bold'

# any scan parameters that you want to filter the API search results by
"""Current possible filters:
   SNR, TSNR, DVAR, FD,
   FWHM, Tesla, gsr_x, gsr_y, TE, TR
   NOTE: Only working as *and* right now!
"""
filter_list = ['TR > 2.0','FD < .3']

# IQM variables to visualize
IQM_to_plot = ['fwhm_avg','fber']

In [3]:
# Load in your own data # 

# This should be a .csv or .tsv file outputted from MRIQC on your own data
# This will return a pandas dataframe of the MRIQC data from your experiment

userdf = load_groupfile(group_file)
# userdf.head()
userdf.shape

/Users/catherinewalsh/Documents/Neurohackademy_2019/mriqception/tools/utils.py:67: FutureWarning:

read_table is deprecated, use read_csv instead, passing sep='\t'.



(142, 45)

In [4]:
# Load and filter data from the API # 

# Figure out which to get from modality arg #
T1apicsv = os.path.join(here, 'demo_api', 'T1w_demo.csv')
T2apicsv = os.path.join(here, 'demo_api', 'T2w_demo.csv')
boldapicsv = os.path.join(here, 'demo_api', 'bold_demo.csv')

if modality == 'T1w':
    api_file = T1apicsv
elif modality == 'T2w':
    api_file = T1apicsv
elif modality == 'bold':
    api_file = boldapicsv

# This will return a pandas dataframe with data from all scans of the given scan type
# with the given parameters 

apidf = pd.read_csv(api_file)
filtered_apidf = filterIQM(apidf,filter_list)
# apidf.head()
print(apidf.shape)
# filtered_apidf.head()
print(filtered_apidf.shape)

(1000, 77)
(203, 77)


In [5]:
# Merge dataframes # 

# Takes the user data and API data and merges it into one dataframe 
# This will return a single pandas dataframe with the local data and API data merged, with a "group" measure to allow for a "groupby" 
# this needs to be updated with actual function name and information about how to use  

vis_ready_df = merge_dfs(userdf, filtered_apidf)
#print(vis_ready_df.head())
#print(vis_ready_df.tail())
vis_ready_df.shape



/Users/catherinewalsh/Documents/Neurohackademy_2019/mriqception/tools/utils.py:90: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



(345, 80)

In [6]:
# Visualization # 

# Create a violin plot of each MRIQC metric, comparing user level-data to API data 
# Also will print out a short description of the measure and what the appropriate range/values should be 
# Input the merged dataframe including the API and user data
# Optional arguments include a list of specific variables that the user wants displayed 
# If no optional arguments included, will plot all measures 

v = make_vio_plot(vis_ready_df, IQM_to_plot)
v


Loading in dataframe...
Loading variables: ['fwhm_avg', 'fber']
(345, 80)
0        0
1        1
2        2
3        3
4        4
5        5
6        6
7        7
8        8
9        9
10      10
11      11
12      12
13      13
14      14
15      15
16      16
17      17
18      18
19      19
20      20
21      21
22      22
23      23
24      24
25      25
26      26
27      27
28      28
29      29
      ... 
642    642
920    920
921    921
924    924
926    926
927    927
929    929
930    930
931    931
932    932
933    933
934    934
935    935
936    936
937    937
938    938
939    939
940    940
941    941
942    942
943    943
945    945
946    946
947    947
948    948
949    949
950    950
951    951
952    952
953    953
Name: _INDEX, Length: 345, dtype: int64


SystemExit: 

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3333: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

